In [46]:
import pandas as pd
import re

lr = "2e4"

aime24 = pd.read_csv(f"SFT/aime24_{lr}_sft.csv")
gpqa = pd.read_csv(f"SFT/gpqa_{lr}_sft.csv")
math500 = pd.read_csv(f"SFT/math500_{lr}_sft.csv")

In [47]:
math500 = math500[["answer","model_response"]]
gpqa = gpqa[["Correct Answer", "shuffled_correct_option", "model_response"]]
aime24 = aime24[["answer","model_response"]]

# Function to get last 15 words
def last_15_words(text):
    return ' '.join(str(text).split()[-35:])

# Apply to each DataFrame
math500['model_response'] = math500['model_response'].apply(last_15_words)
gpqa['model_response'] = gpqa['model_response'].apply(last_15_words)
aime24['model_response'] = aime24['model_response'].apply(last_15_words)


# Save the modified DataFrames to new CSV files
math500.to_csv(f"SFT/math500_{lr}_sf_extracted.csv", index=False)
gpqa.to_csv(f"SFT/gpqa_{lr}_sf_extracted.csv", index=False)
aime24.to_csv(f"SFT/aime24_{lr}_sf_extracted.csv", index=False)

# GPQA Accuracy

In [48]:
def matches(row):
    correct_option = row['shuffled_correct_option'].strip()
    correct_answer = row['Correct Answer'].strip()
    model_answer = str(row['model_response']).strip()

    # Convert to lowercase for case-insensitive search
    lowered = model_answer.lower()
    last_correct_index = lowered.rfind("correct")

    # Slice only the part after the last occurrence of "correct"
    if last_correct_index != -1:
        model_answer = model_answer[last_correct_index + len("correct"):].strip()

    return (
        correct_answer in model_answer or
        f"({correct_option})" in model_answer
    )

matched_count = gpqa.apply(matches, axis=1).sum()
print(f"Number of matching rows: {matched_count}")

Number of matching rows: 54


# AIME24 Accuracy

In [49]:
def extract_numbers(text):
    # Extract integers, decimals, scientific notation, and negative numbers
    return re.findall(r"-?\d+(?:\.\d+)?(?:e[+-]?\d+)?", str(text).lower())

def matches(row):
    answer_numbers = extract_numbers(row['answer'])
    model_numbers = extract_numbers(row['model_response'])

    # Check if all numbers from answer are in model_answer
    return all(num in model_numbers for num in answer_numbers)

matched_count = aime24.apply(matches, axis=1).sum()
print(f"Number of matching rows: {matched_count}")

Number of matching rows: 1


# MATH500

In [50]:
def extract_numbers(text):
    # Extract integers, decimals, scientific notation, and negative numbers
    return re.findall(r"-?\d+(?:\.\d+)?(?:e[+-]?\d+)?", str(text).lower())

def matches(row):
    answer_numbers = extract_numbers(row['answer'])
    model_numbers = extract_numbers(row['model_response'])

    # Check if all numbers from answer are in model_answer
    return all(num in model_numbers for num in answer_numbers)

matched_count = math500.apply(matches, axis=1).sum()
print(f"Number of matching rows: {matched_count}")

Number of matching rows: 176
